In [2]:
import os
import pandas as pd
import numpy as np
from scipy.stats import shapiro, ttest_ind, ks_2samp
import pingouin as pg  # For ICC calculation

# -------------- 读取 CSV --------------
data_dir = './videos/bed_rest_MRI/bed_rest_MRI_L4L5'

# 根据需要选择不同的 prompt_dir
prompt_dir = 'manual_prompt_frame0_1_2'
prompt_dir = 'manual_prompt_frame0_1'
prompt_dir = 'manual_prompt_frame0'

auto_seg_csv = os.path.join(data_dir, prompt_dir, prompt_dir + "fat_ratio_results.csv")
label_seg_csv = os.path.join(data_dir, 'label_72_fat_ratio_results.csv')

auto_df = pd.read_csv(auto_seg_csv)
label_df = pd.read_csv(label_seg_csv)

# -------------- 过滤数据 --------------
manual_prompt_id = ['ID4D_BDC', 'ID12M_R13', 'ID15Q_R13']
remove_prefixes = manual_prompt_id[: 1 + ("1" in prompt_dir) + ("2" in prompt_dir)]
filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]

auto_df, label_df = map(filter_pid, (auto_df, label_df))

# -------------- 确保 PID 和 Class 匹配 --------------
merged_df = auto_df.merge(label_df, on=["PID", "Class"], suffixes=("_auto", "_label"))

# -------------- 计算 MAPE 和 ICC（从 ICC 结果直接提取 95% CI） --------------
mape_results = []
icc_results = []
p_value_dict = {}

for class_value in [1, 2, 3, 4]:
    class_data = merged_df[merged_df["Class"] == class_value]

    if len(class_data) < 2:  # 至少需要 2 组数据进行统计检验
        print(f"Class {class_value}: Skipping due to insufficient samples.")
        continue

    # 提取脂肪比率数据
    auto_values = class_data["Mean Fat Ratio_auto"].values
    label_values = class_data["Mean Fat Ratio_label"].values

    # 计算 MAPE 和标准差
    epsilon = 1e-6
    ape = np.abs((auto_values - label_values) / (label_values + epsilon))
    mape = np.mean(ape)
    mape_std = np.std(ape)
    mape_results.append((class_value, mape, mape_std))
    
    # 计算 ICC
    icc_data = pd.DataFrame({
        'Rater': ['Auto'] * len(auto_values) + ['Label'] * len(label_values),
        'PID': list(class_data["PID"]) * 2,
        'Fat_Ratio': np.concatenate((auto_values, label_values))
    })
    icc_result = pg.intraclass_corr(data=icc_data, targets='PID', raters='Rater', ratings='Fat_Ratio')

    # 提取 ICC2（单个随机评分者）及其 95% 置信区间
    icc_row = icc_result.loc[icc_result['Type'] == 'ICC2']
    icc = icc_row['ICC'].values[0]
    icc_ci_lower, icc_ci_upper = icc_row['CI95%'].values[0][0], icc_row['CI95%'].values[0][1]
    icc_results.append((class_value, icc, icc_ci_lower, icc_ci_upper))

    # -------------- 正态性检验 --------------
    shapiro_label = shapiro(label_values)
    shapiro_auto_seg = shapiro(auto_values)

    label_is_normal = shapiro_label.pvalue > 0.05
    auto_seg_is_normal = shapiro_auto_seg.pvalue > 0.05

    print(f"\nClass {class_value}:")
    print(f"  Shapiro-Wilk Normality Test (Label): p-value = {shapiro_label.pvalue:.3f} -> {'Normal' if label_is_normal else 'Non-Normal'}")
    print(f"  Shapiro-Wilk Normality Test (Auto-Seg): p-value = {shapiro_auto_seg.pvalue:.3f} -> {'Normal' if auto_seg_is_normal else 'Non-Normal'}")

    # -------------- 选择统计检验方法 --------------
    if label_is_normal and auto_seg_is_normal:
        t_stat, p_value = ttest_ind(label_values, auto_values, equal_var=False)
        print("  Method: Independent Two-Sample T-Test")
        print(f"  T-statistic: {t_stat:.4f}, P-value: {p_value:.3f}")
    else:
        ks_stat, p_value = ks_2samp(label_values, auto_values)
        print("  Method: Kolmogorov-Smirnov Test")
        print(f"  KS-statistic: {ks_stat:.4f}, P-value: {p_value:.3f}")
    p_value_dict[class_value] = p_value

    # -------------- 解释检验结果 --------------
    if p_value < 0.05:
        print("  - Significant difference detected (p < 0.05).")
    else:
        print("  - No significant difference detected (p >= 0.05).")



In [8]:

mape_df = pd.DataFrame(mape_results, columns=['Class', 'MAPE', 'MAPE_std'])
icc_df = pd.DataFrame(icc_results, columns=['Class', 'ICC', 'ICC_95%_CI_Lower', 'ICC_95%_CI_Upper'])

# 将 MAPE 及其标准差以百分比格式显示（保留一位小数）
mape_df['MAPE'] = (mape_df['MAPE'] * 100).round(1)
mape_df['MAPE_std'] = (mape_df['MAPE_std'] * 100).round(1)

# 打印最终结果
print("\n### MAPE Results ###")
print(mape_df)

print("\n### ICC Results ###")
print(icc_df)


### MAPE Results ###
   Class  MAPE  MAPE_std
0      1  30.4      36.5
1      2  21.6      15.9
2      3  20.7      13.7
3      4  21.6      33.3

### ICC Results ###
   Class       ICC  ICC_95%_CI_Lower  ICC_95%_CI_Upper
0      1  0.765005              0.41              0.89
1      2  0.686477              0.41              0.83
2      3  0.753262              0.60              0.85
3      4  0.832229              0.74              0.89


In [10]:
import os
import numpy as np
import pandas as pd
from scipy.stats import shapiro, ttest_rel, wilcoxon, ks_1samp, norm
from statsmodels.stats.weightstats import ttost_paired

# -------------- 读取 CSV --------------
data_dir = './videos/bed_rest_MRI/bed_rest_MRI_L4L5'

# 根据需要选择不同的 prompt_dir
prompt_dir = 'manual_prompt_frame0_1_2'
# prompt_dir = 'manual_prompt_frame0_1'
# prompt_dir = 'manual_prompt_frame0'

auto_seg_csv = os.path.join(data_dir, prompt_dir, prompt_dir + "fat_ratio_results.csv")
label_seg_csv = os.path.join(data_dir, 'label_72_fat_ratio_results.csv')

auto_df = pd.read_csv(auto_seg_csv)
label_df = pd.read_csv(label_seg_csv)

# -------------- 过滤数据 --------------
manual_prompt_id = ['ID4D_BDC', 'ID12M_R13', 'ID15Q_R13']
remove_prefixes = manual_prompt_id[: 1 + ("1" in prompt_dir) + ("2" in prompt_dir)]
filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]

auto_df, label_df = map(filter_pid, (auto_df, label_df))

# -------------- 合并数据 --------------
merged_df = auto_df.merge(label_df, on=["PID", "Class"], suffixes=("_auto", "_label"))

# -------------- 辅助函数：最小等效 margin --------------
def find_min_margin(auto, man, method="auto"):
    def tost_p(delta):
        if method == "auto":
            z = (auto - man - np.mean(auto - man)) / np.std(auto - man, ddof=1)
            ks_p = ks_1samp(z, norm.cdf).pvalue
            is_normal = ks_p >= 0.05
        else:
            is_normal = method == "TOST"
        if is_normal:
            p, _, _ = ttost_paired(auto, man, -delta, delta)
        else:
            count = sum(-delta < np.mean(auto[np.random.choice(len(man), len(man), True)] -
                                         man[np.random.choice(len(man), len(man), True)]) < delta for _ in range(1000))
            p = 1 - count / 1000
        return p

    m = np.mean(man)
    low, high = 0.001, 0.5
    for _ in range(20):
        mid = (low + high) / 2
        if tost_p(mid * m) < 0.05:
            high = mid
        else:
            low = mid
        if high - low < 1e-4:
            break
    return round(high, 4)

# -------------- 主循环分析 --------------
label_dict = {1: "Right LES", 2: "Right MF", 3: "Left MF", 4: "Left LES"}
for class_value in [4, 1, 3, 2]:
    class_data = merged_df[merged_df["Class"] == class_value]
    if len(class_data) < 2:
        continue

    auto = class_data["Mean Fat Ratio_auto"].values
    man = class_data["Mean Fat Ratio_label"].values
    diff = auto - man
    label = label_dict[class_value]
    delta = 0.10 * np.mean(man)

    # 正态性判断
    p_normal = shapiro(diff).pvalue
    is_normal = p_normal >= 0.05

    # 差异性检验
    if is_normal:
        p_diff = ttest_rel(auto, man).pvalue
        diff_method = "Paired t-test"
    else:
        try:
            p_diff = wilcoxon(auto, man).pvalue
            diff_method = "Wilcoxon signed-rank test"
        except ValueError:
            p_diff = np.nan
            diff_method = "Wilcoxon test (invalid)"

    # TOST
    z = (diff - np.mean(diff)) / np.std(diff, ddof=1)
    ks_p = ks_1samp(z, norm.cdf).pvalue
    if ks_p >= 0.05:
        p_eq, _, _ = ttost_paired(auto, man, -delta, delta)
        method = "TOST"
    else:
        count = sum(-delta < (auto[idx] - man[idx]).mean() < delta
                    for idx in [np.random.choice(len(man), len(man), True) for _ in range(1000)])
        p_eq = 1 - count / 1000
        method = "Bootstrap"

    # 最小等效 margin
    min_margin = find_min_margin(auto, man, method="auto")

    # === 打印输出 ===
    print(f"{label} (Class {class_value})")
    if p_eq < 0.05:
        print(f"    → ✅ 统计等效（{method}）p = {p_eq:.4f} (margin = 10.0%)")
    else:
        print(f"    → ❌ 不满足统计等效（{method}）p = {p_eq:.4f} (margin = 10.0%)")

    print(f"    → 最小等效 margin ≈ {min_margin:.4f} ({min_margin * 100:.2f}%)")

    if p_diff < 0.05:
        print(f"    → ❌  存在显著差异（{diff_method}）p = {p_diff:.4f}")
    else:
        print(f"    → ✅ 无显著差异（{diff_method}）p = {p_diff:.4f}")

    if p_eq < 0.05 and p_diff >= 0.05:
        print("    ✅✅ 理想情况：等效 且 无显著差异")
    print("\n")


Left LES (Class 4)
    → ❌ 不满足统计等效（TOST）p = 0.4632 (margin = 10.0%)
    → 最小等效 margin ≈ 0.1349 (13.49%)
    → ❌  存在显著差异（Wilcoxon signed-rank test）p = 0.0000


Right LES (Class 1)
    → ✅ 统计等效（TOST）p = 0.0002 (margin = 10.0%)
    → 最小等效 margin ≈ 0.0503 (5.03%)
    → ✅ 无显著差异（Wilcoxon signed-rank test）p = 0.1405
    ✅✅ 理想情况：等效 且 无显著差异


Left MF (Class 3)
    → ✅ 统计等效（TOST）p = 0.0019 (margin = 10.0%)
    → 最小等效 margin ≈ 0.0587 (5.87%)
    → ✅ 无显著差异（Paired t-test）p = 0.8271
    ✅✅ 理想情况：等效 且 无显著差异


Right MF (Class 2)
    → ✅ 统计等效（TOST）p = 0.0416 (margin = 10.0%)
    → 最小等效 margin ≈ 0.0981 (9.81%)
    → ❌  存在显著差异（Paired t-test）p = 0.0041


